# [RDD Programming Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)

## [Overview](https://spark.apache.org/docs/latest/rdd-programming-guide.html#overview)

**High level** her Spark uygulaması ***driver program*** denen bir şeyden oluşur. Bu ***driver program*** kullanıcın *main* fonksiyonunu çalıştırır ve **cluster** üzerinde çeşitli paralel işlemleri execute eder.

Spark' ın sağladığı main **absraction** ***resilient distributed dataset*** (RDD) olmaktadır. RDD, bir elemanlar kümesidir(collection of elements). RDD, cluster üzerindeki node' lar boyunca partition' lara (bölümlere) ayrılmıştır. RDD' lerin node' lara bölünmesi sayesinde paralel işlemler yapılabilmektedir.

RDD' ler çeşitli formattaki dosyalardan ya da driver program içindeki collectionlardan oluşturulabilir (collectiondan kasıt sanırım scala dizileri ya da hangi dili kullanıyorsanız).

Eğer bir RDD' yi çok sık şekilde kullanacaksanız Spark' a bu RDD' yi cache al diyebiliyorsunuz.

RDD' ler node' larda oluşan hatalardan otomatik olarak recover edilirler Sanırım hiç hata olmamış hallerine geri döndürülürler.

___

Spark' ın ikinci absraction' u paralel işlemler kullanılan ***shared variables*** tir.

Sanırım Spark' ta Default olarak şöyle bir şey gerçekleştiriliyor: Spark' ta paralel bir işlem başladığında node' lara task' lar dağıtılıyor. Bu task' lar için gerekli olan variable' lar node' larda çalışacak olan fonksiyona kopyalanıyor.

Bazen bir tane değişkenin tasklar arasında ya da task ile driver program arasında shared edilmesi gerekiyormuş. Bu özelliğin var olma sebebi bu.

Sparkta iki tane shared variables türü varmış:
***broadcast variables***: Tüm nodeların memory' lerindeki bir value' yi cache almak için kullanılır. (Ne demek bilmiyorum.)
***accumulators***: Yanlızca "added" değişkenlermiş. Örneğin counters ve sum. (Ne demek bilmiyorum as well)

## [Resilient Distributed Datasets (RDDs)](https://spark.apache.org/docs/latest/rdd-programming-guide.html#resilient-distributed-datasets-rdds)

Spark ***Resilient Distributed Datasets*** kavramı etrafında döner. RDD, paralel olarak çalıştırılabilen, hata toleranslı elementler koleksiyonudur (**fault-tolerant** collection of elements).

RDD iki şekilde oluşturulur: <br/>
birincisi driver programda bulunan colloection' u ***parallelizing*** ederek. <br/>
ikincisi external storage system' da bulunan bir dataset' e referans vererek. external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.


### [Parallelized Collections](https://spark.apache.org/docs/latest/rdd-programming-guide.html#parallelized-collections)

Bir parallelized collection, sizin driver programınızda bulunan collection' ı parametre olarak olan ***SparkContext***' in ***parallelize*** metodu ile oluşturulur.

In [2]:
val data = Array(1, 2, 3, 4, 5)
val distData = sc.parallelize(data) //RDD oluştu Normally, Spark tries to set the number of partitions automatically based on your cluster. However, you can also set it manually by passing it as a second parameter to parruldu 
// ve paralel işlem yapılabilir artık.

data: Array[Int] = Array(1, 2, 3, 4, 5)
distData: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at <console>:26


Parallel collection' larda önemli bir konuda bu RDD' nin kaç tane partition' a bölüneceği. Spark normalde partion' lama işlemlerini kendi yaparmış. Ama elle biz de verebilirmişiz. Elle vermek istersek `sc.parallelize(data, 10)` bu şekilde yapabiliriz.

> Bazı yerlerde `partions` yerine `slices` ifadesi geçebilirmiş.

## [External Datasets](https://spark.apache.org/docs/latest/rdd-programming-guide.html#external-datasets)

Spark can create distributed datasets from any storage source supported by Hadoop, including your local file system, HDFS, Cassandra, HBase, Amazon S3, etc. Spark supports text files, SequenceFiles, and any other Hadoop InputFormat.

Text file RDDs can be created using ***SparkContext***’s ***textFile*** method. This method takes a URI for the file (either a local path on the machine, or a hdfs://, s3a://, etc URI) and reads it as a collection of lines. Here is an example invocation:

In [2]:
import org.apache.spark.rdd.RDD;

val distFİle: RDD[String] = sc.textFile("../README.md") //distFile' in türünü atamasını Scala' nın bulması yerine
// kendim atadığım için compile edilirken daha hızlı çalışacaktır.

import org.apache.spark.rdd.RDD
distFİle: org.apache.spark.rdd.RDD[String] = ../README.md MapPartitionsRDD[1] at textFile at <console>:27


Some notes on reading files with Spark:
- Bir tane ana bilgisayar var ve ona bağlı köle bilgisayarlar (worker node) var. Ana bilgisayarın local' ındeki file' a köle bilgisayarlar tabiki de erişemeyecek. Erişebilmesi için ya dosyayayı workerlara kopyala ya da internet üzerinden erişmelerini sağla
- All of Spark’s file-based input methods, including textFile, support running on directories, compressed files, and wildcards as well. For example, you can use textFile("/my/directory"), textFile("/my/directory/*.txt"), and textFile("/my/directory/*.gz").
- `textFile` metoduna isteğe bağlı RDD' nin partition' ını belirlemek için ikinci bir parametre daha verebiliyorsun. Default olarak bu partition' lara bölme işlemini `Spark zaten dosyadaki her bir block için bir tane partition oluşturarak yapıyor (HDFS' te her bir block 128MB' tır)`. İşte textFile metodunun ikinci parametresi sayesinde bu sayıyı arttırabiliyoruz. block değerinden aşağı bir partition sayısı belirleyemiyoruz. Default block size: 128MB. block size 256MB yapılamaz. `Default partition değeri 5 ise bu değeri parametre ile 4 yapamazsın. Çünkü partiton size' ı 128MB' den büyük olur.`

> Devamıda vardı yazmadım. İlgili olan başlığa tıklayarak gerekli linke gidebilir.

## [RDD Operations](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-operations)

RDD' ler iki tür işleme destek verirler:
- **transformation**: var olan bir RDD' den yeni bir RDD elde etme.
- **actions**: Bir RDD' den belli hesaplamalar ile driver program' a bir değer döndürülmesi işlemleri.

Transformation işlemleri ile oluşan RDD' ler hemen oluşturulmazlarmış(transformation işlemleri saklanırmış). Eğer bir action gelirse RDD oluşturulur ve daha sonra action işlemi gerçekleştirilirmiş. Bu Spark' a verimlilik katarmış (Ben ikna olmadım).

Default olarak her action işlemi ile RDD oluşur ve üzerinden işlem yapılırmış ama istersek oluşan RDD' yi ***persist*** hale getirebiliyormuşuz: `action` ya da `persist` metodları ile. Eğer RDD üzerinde çok işlem yapılacaksa persist hale getirmek verimliliği arttır. (Buna ikna oldum)

### [Basic](https://spark.apache.org/docs/latest/rdd-programming-guide.html#basics)

